In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import random
import numpy as np
import tqdm

Data

In [2]:
# fig, axs = plt.subplots()
# result['creation_year'].hist(bins=40,range=(0,2000))
# plt.title('Frequency of paintings per timeperiod')
# plt.xlabel('Year')
# plt.ylabel('Frequency')

## Create dataset with needed columns

In [3]:
# Read metalevelmeta data
data_level = pd.read_csv('omniart_data/metalevelmeta.csv')
data_level.set_index('omni_id', inplace=True)
data_level = data_level.loc[~data_level.index.duplicated(keep='first')] # Remove duplicates
# Read metadatas
data_meta = pd.read_csv('omniart_data/metadatas.csv')
data_meta.set_index('omni_id', inplace=True)
data_meta = data_meta.loc[~data_meta.index.duplicated(keep='first')]
# Concatenate useful columns
result = pd.concat([data_level['textual_description'], data_level['iconclass_codes'], data_level['color_pallete'],
                    data_level['palette_count'], data_level['dominant_color'], data_level['concepts'],
                   data_meta['artwork_name'], data_meta['artist_full_name'], data_meta['creation_year'],
                   data_meta['source_url'], data_meta['image_url'], data_meta['school']], axis=1)


C:\Users\linda\Miniconda3\envs\env_torch\lib\site-packages\IPython\core\interactiveshell.py:3051: DtypeWarning: Columns (3,4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\linda\Miniconda3\envs\env_torch\lib\site-packages\IPython\core\interactiveshell.py:3051: DtypeWarning: Columns (5,6,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
resultbig = result

In [5]:
# Set creation year to integers and remove unknown/nan values
result = result[result['creation_year'].notna()]
result = result[result['creation_year']!='unknown']
result["creation_year"] = result["creation_year"].astype(float)
result["creation_year"] = result["creation_year"].astype(int)
# Remove values with unkown/nan dominant colors
result = result[result['dominant_color'].notna()]
result = result[result['dominant_color']!='unknown']

result

,textual_description,iconclass_codes,color_pallete,palette_count,dominant_color,concepts,artwork_name,artist_full_name,creation_year,source_url,image_url,school
omni_id,,,,,,,,,,,,
0,NaN,NaN,"[u'#a27967', u'#786b60', u'#704945', u'#251f22...","[5549, 6251, 5073, 6659, 3610, 5542, 6337, 469...",#251f22,NaN,allegory,hans von aachen,1598,https://www.wga.hu/html/a/aachen/allegory.html,http://www.wga.hu/art/a/aachen/allegory.jpg,"Deutschland, Europe"
1,NaN,NaN,"[u'#f2dfc3', u'#dbb38c', u'#a2775f', u'#90574a...","[4153, 3961, 3795, 3848, 5379, 4799, 5860, 723...",#1b1619,NaN,"bacchus, ceres and cupid",hans von aachen,1625,https://www.wga.hu/html/a/aachen/bacchus.html,http://www.wga.hu/art/a/aachen/bacchus.jpg,german
2,NaN,NaN,"[u'#b68761', u'#7b4934', u'#5d3d30', u'#412f28...","[3705, 4051, 5450, 6136, 6692, 5925, 6016, 567...",#1f1a1a,NaN,joking couple,hans von aachen,1625,https://www.wga.hu/html/a/aachen/j_couple.html,http://www.wga.hu/art/a/aachen/j_couple.jpg,"Deutschland, Europe"
3,NaN,NaN,"[u'#cfa57c', u'#322526', u'#2d1d1e', u'#1e191d...","[3905, 4731, 4422, 6034, 4961, 8425, 4790, 359...",#0e0d12,NaN,portrait of emperor rudolf ii,hans von aachen,1590,https://www.wga.hu/html/a/aachen/rudolf2.html,http://www.wga.hu/art/a/aachen/rudolf2.jpg,"Deutschland, Europe"
4,NaN,NaN,"[u'#7d3f1d', u'#795325', u'#6f3516', u'#372d1b...","[4769, 4493, 5274, 4275, 6177, 7510, 3649, 346...",#28241a,NaN,self-portrait with a glass of wine,hans von aachen,1596,https://www.wga.hu/html/a/aachen/selfport.html,http://www.wga.hu/art/a/aachen/selfport.jpg,"Deutschland, Europe"
...,...,...,...,...,...,...,...,...,...,...,...,...
14942784,NaN,NaN,"[u'#f9f7dc', u'#f6f1d3', u'#f3edd0', u'#eeebcc...","[4618, 7647, 5150, 4582, 4968, 7417, 5036, 589...",#f6f1d3,NaN,image taken from page 483 of '[the works of be...,"cunningham, francis - lieut.-colonel",1875,https://www.flickr.com/photos/britishlibrary/w...,http://farm6.staticflickr.com/5530/11064767236...,london
14942786,NaN,NaN,"[u'#fffffc', u'#fefff9', u'#fefff3', u'#fdfef5...","[5248, 9963, 7435, 2073, 4819, 6023, 4920, 266...",#fefff9,NaN,image taken from page 16 of 'auf biblischen pf...,"ninck, c.",1885,https://www.flickr.com/photos/britishlibrary/w...,http://farm4.staticflickr.com/3692/11287246996...,hamburg
14942787,NaN,NaN,"[u'#fefef9', u'#514f4a', u'#3e3d3b', u'#fefee7...","[7084, 5067, 3226, 10051, 4763, 3709, 6385, 51...",#fefee7,NaN,image taken from page 513 of 'kulturgeschichte...,"brandt, max von.",1896,https://www.flickr.com/photos/britishlibrary/w...,http://farm4.staticflickr.com/3721/11238734035...,leipzig


Create country dict

In [6]:
# Create dict of {id: }
country_dict = {}
filename = 'preprocessing/ctry_ids.txt'
with open(filename, 'rb') as f:
    for line in f:
        var = line.strip()[0:3]
        ctr_id = var.decode('UTF-8')
        country_dict[line.strip()[4:].decode('UTF-8')] = ctr_id

        
# # Use other dataset as well: https://gist.github.com/zspine/2365808#file-countries-csv
# countries = pd.read_csv('countries.csv')
# for country in list(list(countries['Name'])):
#     country_dict[country] = countries[countries['Name']==country]['CCA3'].values[0]

country_dict_inv = {country_dict[country]:country for country in list(country_dict)}


Add countries from other dataset and make sure they are in the same format using replace_dict1

In [8]:

# Use other dataset as well: https://gist.github.com/zspine/2365808#file-countries-csv
country_values = [country_dict[key] for key in list(country_dict)]
country_keys = list(country_dict)

replace_dict1 = {}
countries = pd.read_csv('preprocessing/countries.csv')
for country in list(list(countries['Name'])):
    if country not in country_keys:
        country_code = countries[countries['Name']==country]['CCA3'].values[0]
        if country_code not in country_values:
            country_dict[country] = country_code
#             print(country_dict[country], country_code)
        else:
            replace_dict1[country] = country_dict_inv[country_code]
#     country_dict[country] = countries[countries['Name']==country]['CCA3'].values[0]
replace_dict1

{'Bahamas': 'The Bahamas',
 'Congo - Kinshasa': 'Democratic Republic of the Congo',
 'Congo - Brazzaville': 'Republic of the Congo',
 'Côte d’Ivoire': 'Ivory Coast',
 'French Southern and Antarctic Territories': 'French Southern and Antarctic Lands',
 'Guinea-Bissau': 'Guinea Bissau',
 'Myanmar [Burma]': 'Myanmar',
 'Palestinian Territories': 'West Bank',
 'Serbia': 'Republic of Serbia',
 'French Southern Territories': 'French Southern and Antarctic Lands',
 'Timor-Leste': 'East Timor',
 'Tanzania': 'United Republic of Tanzania',
 'United States': 'United States of America'}

Preprocess for frequencies >4000

In [9]:
country_dict

{'Afghanistan': 'AFG',
 'Angola': 'AGO',
 'Albania': 'ALB',
 'United Arab Emirates': 'ARE',
 'Argentina': 'ARG',
 'Armenia': 'ARM',
 'Antarctica': 'ATA',
 'French Southern and Antarctic Lands': 'ATF',
 'Australia': 'AUS',
 'Austria': 'AUT',
 'Azerbaijan': 'AZE',
 'Burundi': 'BDI',
 'Belgium': 'BEL',
 'Benin': 'BEN',
 'Burkina Faso': 'BFA',
 'Bangladesh': 'BGD',
 'Bulgaria': 'BGR',
 'The Bahamas': 'BHS',
 'Bosnia and Herzegovina': 'BIH',
 'Belarus': 'BLR',
 'Belize': 'BLZ',
 'Bolivia': 'BOL',
 'Brazil': 'BRA',
 'Brunei': 'BRN',
 'Bhutan': 'BTN',
 'Botswana': 'BWA',
 'Central African Republic': 'CAF',
 'Canada': 'CAN',
 'Switzerland': 'CHE',
 'Chile': 'CHL',
 'China': 'CHN',
 'Ivory Coast': 'CIV',
 'Cameroon': 'CMR',
 'Democratic Republic of the Congo': 'COD',
 'Republic of the Congo': 'COG',
 'Colombia': 'COL',
 'Costa Rica': 'CRI',
 'Cuba': 'CUB',
 'Northern Cyprus': '-99',
 'Cyprus': 'CYP',
 'Czech Republic': 'CZE',
 'Germany': 'DEU',
 'Djibouti': 'DJI',
 'Denmark': 'DNK',
 'Dominican

In [10]:
result['school'] = result['school'].str.lower()
freqs = dict(result['school'].value_counts())

In [11]:
freqs = dict(result['school'].value_counts())
# freqs
# for country in list(freqs):
#     print(country, freqs[country])

In [12]:
# For all with freq > 4000
replace_dict1['birmingham'] = 'United Kingdom'
replace_dict1['iran'] = 'Iran'
replace_dict1['melbourne'] = 'Australia'
replace_dict1['english'] = 'United Kingdom'
replace_dict1['firenze'] = 'Italy'
replace_dict1['calcutta'] = 'India'
replace_dict1['lausanne'] = 'Switzerland'
replace_dict1['london]'] = 'United Kingdom'

replace_dict1['london & edinburgh'] = 'United Kingdom'
replace_dict1['strassburg'] = 'France'
replace_dict1['mexico'] = 'Mexico'
replace_dict1['scottish'] = 'United Kingdom'
replace_dict1['new-york'] = 'United States of America'
replace_dict1['london, glasgow [printed]'] = 'United Kingdom'
replace_dict1['india'] = 'India'


replace_dict1['wurzburg'] = 'Germany'
replace_dict1['san francisco'] = 'United States of America'
replace_dict1['leiden'] = 'Netherlands'
replace_dict1['prag'] = 'Czech Republic'
replace_dict1['edinburgh & london'] = 'United Kingdom'
replace_dict1['rouen'] = 'France'
replace_dict1['london, edinburgh [printed]'] = 'United Kingdom'
replace_dict1['oxford'] = 'United Kingdom'


replace_dict1['kiel'] = 'Germany'
replace_dict1['4 vol. the century co. new york, 1887, 8'] = 'United States of America'
replace_dict1['london; edinburgh [printed]'] = 'United Kingdom'
replace_dict1['boston, mass'] = 'United States of America'
replace_dict1['bristol'] = 'United Kingdom'
replace_dict1['washington'] = 'United States of America'

replace_dict1['lisboa'] = 'Portugal'
replace_dict1['kbenhavn'] = 'Denmark'
replace_dict1['london, guildford [printed]'] = 'United Kingdom'
replace_dict1['tours'] = 'France'
replace_dict1['hartford, conn'] = 'United States of America'
replace_dict1['zurich'] = 'Switzerland'
replace_dict1['kristiania'] = 'Norway'
replace_dict1['cincinnati'] = 'United States of America'


replace_dict1['spanish'] = 'Spain'
replace_dict1['dublin'] = 'Ireland'
replace_dict1['leeds'] = 'United Kingdom'
replace_dict1['london; guildford [printed]'] = 'United Kingdom'
replace_dict1['munchen'] = 'Germany'
replace_dict1['brighton'] = 'United Kingdom'
replace_dict1['cypriot'] = 'Cyprus'


replace_dict1['toronto'] = 'Canada'
replace_dict1['greek'] = 'Greece'
replace_dict1['lyon'] = 'France'
replace_dict1['manchester'] = 'United Kingdom'
replace_dict1['italia'] = 'Italy'
replace_dict1['london, etc'] = 'United Kingdom'
replace_dict1['barcelona'] = 'Spain'


replace_dict1['stockholm'] = 'Sweden'
replace_dict1['bruxelles'] = 'Belgium'
replace_dict1['dutch'] = 'Netherlands'
replace_dict1['boston'] = 'United States of America'
replace_dict1['glasgow'] = 'United Kingdom'
replace_dict1['amsterdam'] = 'Netherlands'
replace_dict1['crewe, manchester'] = 'United Kingdom'
# replace_dict1['flemish'] = 'Austria']

replace_dict1['kjbenhavn'] = 'Denmark'
replace_dict1['madrid'] = 'Spain'
replace_dict1['crewe, manchester'] = 'United Kingdom'
replace_dict1['boston [mass.]'] = 'United States of America'
replace_dict1['stuttgart'] = 'Germany'


replace_dict1['edinburgh'] = 'United Kingdom'
replace_dict1['chicago'] = 'United States of America'
replace_dict1['american'] = 'United States of America'
replace_dict1['milano'] = 'Italy'
replace_dict1['chinese'] = 'China'
replace_dict1['budapest'] = 'Hungary'
replace_dict1['berlin'] = 'Germany'
replace_dict1['wien'] = 'Austria'


replace_dict1['london'] = 'United Kingdom'
replace_dict1['paris'] = 'France'
replace_dict1['new york'] = 'United States of America'
replace_dict1['british'] = 'United Kingdom'
replace_dict1['french'] = 'France'
replace_dict1['philadelphia '] = 'United States of America'
replace_dict1['japanese'] = 'Japan'
replace_dict1['leipzig'] = 'Germany'

replace_dict1['german'] = 'Germany'
replace_dict1['danish'] = 'Denmark'
replace_dict1['belgian'] = 'Belgium'
replace_dict1['españa'] = 'Spain'
replace_dict1['deutschland, europe'] = 'Germany'
replace_dict1['italia'] = 'Italy'
replace_dict1['italian'] = 'Italy'
replace_dict1['nederland'] = 'Netherlands'



In [13]:
result['school'].unique()

array(['deutschland, europe', 'german', 'danmark', ...,
       'pp. x. 429. john murray london, 1894', '] paris', 'gateshead'],
      dtype=object)

In [14]:
replace_dict1

{'Bahamas': 'The Bahamas',
 'Congo - Kinshasa': 'Democratic Republic of the Congo',
 'Congo - Brazzaville': 'Republic of the Congo',
 'Côte d’Ivoire': 'Ivory Coast',
 'French Southern and Antarctic Territories': 'French Southern and Antarctic Lands',
 'Guinea-Bissau': 'Guinea Bissau',
 'Myanmar [Burma]': 'Myanmar',
 'Palestinian Territories': 'West Bank',
 'Serbia': 'Republic of Serbia',
 'French Southern Territories': 'French Southern and Antarctic Lands',
 'Timor-Leste': 'East Timor',
 'Tanzania': 'United Republic of Tanzania',
 'United States': 'United States of America',
 'birmingham': 'United Kingdom',
 'iran': 'Iran',
 'melbourne': 'Australia',
 'english': 'United Kingdom',
 'firenze': 'Italy',
 'calcutta': 'India',
 'lausanne': 'Switzerland',
 'london]': 'United Kingdom',
 'london & edinburgh': 'United Kingdom',
 'strassburg': 'France',
 'mexico': 'Mexico',
 'scottish': 'United Kingdom',
 'new-york': 'United States of America',
 'london, glasgow [printed]': 'United Kingdom',
 'i

Remove cases

In [15]:
country_dict_lower = {}
for country in list(country_dict):
    country_dict_lower[country.lower()] = country_dict[country]
country_dict_lower

{'afghanistan': 'AFG',
 'angola': 'AGO',
 'albania': 'ALB',
 'united arab emirates': 'ARE',
 'argentina': 'ARG',
 'armenia': 'ARM',
 'antarctica': 'ATA',
 'french southern and antarctic lands': 'ATF',
 'australia': 'AUS',
 'austria': 'AUT',
 'azerbaijan': 'AZE',
 'burundi': 'BDI',
 'belgium': 'BEL',
 'benin': 'BEN',
 'burkina faso': 'BFA',
 'bangladesh': 'BGD',
 'bulgaria': 'BGR',
 'the bahamas': 'BHS',
 'bosnia and herzegovina': 'BIH',
 'belarus': 'BLR',
 'belize': 'BLZ',
 'bolivia': 'BOL',
 'brazil': 'BRA',
 'brunei': 'BRN',
 'bhutan': 'BTN',
 'botswana': 'BWA',
 'central african republic': 'CAF',
 'canada': 'CAN',
 'switzerland': 'CHE',
 'chile': 'CHL',
 'china': 'CHN',
 'ivory coast': 'CIV',
 'cameroon': 'CMR',
 'democratic republic of the congo': 'COD',
 'republic of the congo': 'COG',
 'colombia': 'COL',
 'costa rica': 'CRI',
 'cuba': 'CUB',
 'northern cyprus': '-99',
 'cyprus': 'CYP',
 'czech republic': 'CZE',
 'germany': 'DEU',
 'djibouti': 'DJI',
 'denmark': 'DNK',
 'dominican

Set new columns

In [16]:
# Setup country iwht lowercased school
result["country"] = result["school"].str.lower()
print(len(result['country'].unique()))
# Replace schools to country names
result = result.replace({"country": replace_dict1}) 
print(len(result['country'].unique()))

# Lowercase the countries
result["country"] = result["country"].str.lower()
# Setup country id
result["ctry_id"] = result["country"]
result['country'].unique()

4421
4363


array(['germany', 'danmark', 'spain', ...,
       'pp. x. 429. john murray london, 1894', '] paris', 'gateshead'],
      dtype=object)

In [17]:
result['country'].unique()

array(['germany', 'danmark', 'spain', ...,
       'pp. x. 429. john murray london, 1894', '] paris', 'gateshead'],
      dtype=object)

In [18]:
# Remove entries with countries not in the dict
countries = list(country_dict_lower)
result = result[result['country'].isin(countries)]

# Put country ids in ctry_id 
result = result.replace({"ctry_id": country_dict_lower}) 

result

,textual_description,iconclass_codes,color_pallete,palette_count,dominant_color,concepts,artwork_name,artist_full_name,creation_year,source_url,image_url,school,country,ctry_id
omni_id,,,,,,,,,,,,,,
0,NaN,NaN,"[u'#a27967', u'#786b60', u'#704945', u'#251f22...","[5549, 6251, 5073, 6659, 3610, 5542, 6337, 469...",#251f22,NaN,allegory,hans von aachen,1598,https://www.wga.hu/html/a/aachen/allegory.html,http://www.wga.hu/art/a/aachen/allegory.jpg,"deutschland, europe",germany,DEU
1,NaN,NaN,"[u'#f2dfc3', u'#dbb38c', u'#a2775f', u'#90574a...","[4153, 3961, 3795, 3848, 5379, 4799, 5860, 723...",#1b1619,NaN,"bacchus, ceres and cupid",hans von aachen,1625,https://www.wga.hu/html/a/aachen/bacchus.html,http://www.wga.hu/art/a/aachen/bacchus.jpg,german,germany,DEU
2,NaN,NaN,"[u'#b68761', u'#7b4934', u'#5d3d30', u'#412f28...","[3705, 4051, 5450, 6136, 6692, 5925, 6016, 567...",#1f1a1a,NaN,joking couple,hans von aachen,1625,https://www.wga.hu/html/a/aachen/j_couple.html,http://www.wga.hu/art/a/aachen/j_couple.jpg,"deutschland, europe",germany,DEU
3,NaN,NaN,"[u'#cfa57c', u'#322526', u'#2d1d1e', u'#1e191d...","[3905, 4731, 4422, 6034, 4961, 8425, 4790, 359...",#0e0d12,NaN,portrait of emperor rudolf ii,hans von aachen,1590,https://www.wga.hu/html/a/aachen/rudolf2.html,http://www.wga.hu/art/a/aachen/rudolf2.jpg,"deutschland, europe",germany,DEU
4,NaN,NaN,"[u'#7d3f1d', u'#795325', u'#6f3516', u'#372d1b...","[4769, 4493, 5274, 4275, 6177, 7510, 3649, 346...",#28241a,NaN,self-portrait with a glass of wine,hans von aachen,1596,https://www.wga.hu/html/a/aachen/selfport.html,http://www.wga.hu/art/a/aachen/selfport.jpg,"deutschland, europe",germany,DEU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14942783,NaN,NaN,"[u'#e9ddbb', u'#e7d9b7', u'#e3d5b4', u'#e0d5b4...","[4933, 4675, 5579, 2943, 4162, 7756, 5596, 555...",#b2ac92,NaN,image taken from page 239 of 'the hudson ... i...,"bruce, wallace.",1894,https://www.flickr.com/photos/britishlibrary/w...,http://farm3.staticflickr.com/2866/11226156985...,edinburgh,united kingdom,GBR
14942784,NaN,NaN,"[u'#f9f7dc', u'#f6f1d3', u'#f3edd0', u'#eeebcc...","[4618, 7647, 5150, 4582, 4968, 7417, 5036, 589...",#f6f1d3,NaN,image taken from page 483 of '[the works of be...,"cunningham, francis - lieut.-colonel",1875,https://www.flickr.com/photos/britishlibrary/w...,http://farm6.staticflickr.com/5530/11064767236...,london,united kingdom,GBR
14942787,NaN,NaN,"[u'#fefef9', u'#514f4a', u'#3e3d3b', u'#fefee7...","[7084, 5067, 3226, 10051, 4763, 3709, 6385, 51...",#fefee7,NaN,image taken from page 513 of 'kulturgeschichte...,"brandt, max von.",1896,https://www.flickr.com/photos/britishlibrary/w...,http://farm4.staticflickr.com/3721/11238734035...,leipzig,germany,DEU


In [21]:
len(result['artist_full_name'].unique()), len(result['country'].unique()), len(result['ctry_id'].unique()), len(result['dominant_color'].unique())

(23161, 61, 61, 158094)

In [20]:
result.to_csv('schooldata_f1000.csv')

## Country by artist birthplace (DID NOT WORK)

In [2]:
# Read metalevelmeta data
data_level = pd.read_csv('omniart_data/metalevelmeta.csv')
data_level.set_index('omni_id', inplace=True)
data_level = data_level.loc[~data_level.index.duplicated(keep='first')] # Remove duplicates
# Read metadatas
data_meta = pd.read_csv('omniart_data/metadatas.csv')
data_meta.set_index('omni_id', inplace=True)
data_meta = data_meta.loc[~data_meta.index.duplicated(keep='first')]
# Concatenate useful columns
result = pd.concat([data_level['textual_description'], data_level['iconclass_codes'], data_level['color_pallete'],
                    data_level['palette_count'], data_level['dominant_color'], data_level['concepts'],
                   data_meta['artwork_name'], data_meta['artist_full_name'], data_meta['creation_year'],
                   data_meta['source_url'], data_meta['image_url'], data_meta['school']], axis=1)
resultbig = result

C:\Users\linda\Miniconda3\envs\env_torch\lib\site-packages\IPython\core\interactiveshell.py:3051: DtypeWarning: Columns (3,4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\linda\Miniconda3\envs\env_torch\lib\site-packages\IPython\core\interactiveshell.py:3051: DtypeWarning: Columns (5,6,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Remove data with no iconclasses

In [65]:
data = resultbig
data = data[data['iconclass_codes'].notna()]
data['artist_full_name'] = data['artist_full_name'].str.strip()
# data['school'].unique()
data.iloc[0]

C:\Users\linda\Miniconda3\envs\env_torch\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


textual_description      thorns lie on the matress of a four-poster bed.
iconclass_codes                   41A421, 25G(+223), 41A76122, 25G(+222)
color_pallete          [u'#ded3bb', u'#c4b99f', u'#b0a386', u'#9a8d70...
palette_count          [3971, 4754, 6027, 5343, 5033, 8688, 7471, 364...
dominant_color                                                   #5d5135
concepts                                                             NaN
artwork_name                     emblem pictura - patientia vincit omnia
artist_full_name                                   georgette de montenay
creation_year                                                       1584
source_url             http://arkyves.org/view/IIHIM_GEDP_-1937878903...
image_url                       http://arkyves.org/f/IIHIM_498061126.jpg
school                                                           unknown
Name: 13000000, dtype: object

Create country_dict containing two types of country data

In [4]:
# Create dict of {id: }
country_dict = {}
filename = 'preprocessing/ctry_ids.txt'
with open(filename, 'rb') as f:
    for line in f:
        var = line.strip()[0:3]
        ctr_id = var.decode('UTF-8')
        country_dict[line.strip()[4:].decode('UTF-8')] = ctr_id

        
# Use other dataset as well: https://gist.github.com/zspine/2365808#file-countries-csv
countries = pd.read_csv('countries.csv')
for country in list(list(countries['Name'])):
    country_dict[country] = countries[countries['Name']==country]['CCA3'].values[0]
    
# Lower the countries
country_dict_lower = {}
for country in list(country_dict):
    country_dict_lower[country.lower()] = country_dict[country]
country_dict_lower

{'afghanistan': 'AFG',
 'angola': 'AGO',
 'albania': 'ALB',
 'united arab emirates': 'ARE',
 'argentina': 'ARG',
 'armenia': 'ARM',
 'antarctica': 'ATA',
 'french southern and antarctic lands': 'ATF',
 'australia': 'AUS',
 'austria': 'AUT',
 'azerbaijan': 'AZE',
 'burundi': 'BDI',
 'belgium': 'BEL',
 'benin': 'BEN',
 'burkina faso': 'BFA',
 'bangladesh': 'BGD',
 'bulgaria': 'BGR',
 'the bahamas': 'BHS',
 'bosnia and herzegovina': 'BIH',
 'belarus': 'BLR',
 'belize': 'BLZ',
 'bolivia': 'BOL',
 'brazil': 'BRA',
 'brunei': 'BRN',
 'bhutan': 'BTN',
 'botswana': 'BWA',
 'central african republic': 'CAF',
 'canada': 'CAN',
 'switzerland': 'CHE',
 'chile': 'CHL',
 'china': 'CHN',
 'ivory coast': 'CIV',
 'cameroon': 'CMR',
 'democratic republic of the congo': 'COD',
 'republic of the congo': 'COG',
 'colombia': 'COL',
 'costa rica': 'CRI',
 'cuba': 'CUB',
 'northern cyprus': '-99',
 'cyprus': 'CYP',
 'czech republic': 'CZE',
 'germany': 'DEU',
 'djibouti': 'DJI',
 'denmark': 'DNK',
 'dominican

Load moma artist dataset

In [5]:
MoMa_artists = pd.read_csv('Artists.csv')
MoMa_artists['DisplayName'] = MoMa_artists['DisplayName'].str.lower()
MoMa_artists['DisplayName'] = MoMa_artists['DisplayName'].str.strip()

Create dict {artist: nationality}

In [6]:
moma_names = list(MoMa_artists['DisplayName'])
counter = 0
moma_natio = {}
data['artist_full_name'] = data['artist_full_name'].str.strip()
for name in data['artist_full_name'].unique():
    if name in moma_names:
        moma_natio[name] = str(MoMa_artists[MoMa_artists['DisplayName'] == name]['Nationality'].values[0]).lower()
        counter += 1
print(counter)
moma_natio

C:\Users\linda\Miniconda3\envs\env_torch\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


0


{}

Load tate artist dataset

In [53]:
tate_artists = pd.read_csv('artist_data_tate.csv')
# MoMa_artists['DisplayName'] = MoMa_artists['DisplayName'].str.lower()
# MoMa_artists['DisplayName'] = MoMa_artists['DisplayName'].str.strip()
# tate_artists['name_split'] = tate_artists['name'].str.split(',')
tate_artists['name'] = tate_artists['name'].str.split(',').str[1]+' '+tate_artists['name'].str.split(',').str[0]
tate_artists['name'] = tate_artists['name'].str.lower()
tate_artists

,id,name,gender,dates,yearOfBirth,yearOfDeath,placeOfBirth,placeOfDeath,url
0,10093,magdalena abakanowicz,Female,born 1930,1930.0,NaN,Polska,NaN,http://www.tate.org.uk/art/artists/magdalena-a...
1,0,edwin austin abbey,Male,1852–1911,1852.0,1911.0,"Philadelphia, United States","London, United Kingdom",http://www.tate.org.uk/art/artists/edwin-austi...
2,2756,berenice abbott,Female,1898–1991,1898.0,1991.0,"Springfield, United States","Monson, United States",http://www.tate.org.uk/art/artists/berenice-ab...
3,1,lemuel francis abbott,Male,1760–1803,1760.0,1803.0,"Leicestershire, United Kingdom","London, United Kingdom",http://www.tate.org.uk/art/artists/lemuel-fran...
4,622,ivor abrahams,Male,born 1935,1935.0,NaN,"Wigan, United Kingdom",NaN,http://www.tate.org.uk/art/artists/ivor-abraha...
...,...,...,...,...,...,...,...,...,...
3527,12542,gilberto zorio,Male,born 1944,1944.0,NaN,"Andorno Micca, Italia",NaN,http://www.tate.org.uk/art/artists/gilberto-zo...
3528,2186,larry zox,Male,born 1936,1936.0,NaN,"Des Moines, United States",NaN,http://www.tate.org.uk/art/artists/larry-zox-2186
3529,621,francesco zuccarelli,Male,1702–1788,1702.0,1788.0,Italia,Firenze,http://www.tate.org.uk/art/artists/francesco-z...
3530,2187,ignacio zuloaga,Male,1870–1945,1870.0,1945.0,España,España,http://www.tate.org.uk/art/artists/ignacio-zul...


Create dict {artist: nationality}

In [54]:
tate_names = list(tate_artists['name'])
counter = 0
tate_natio = {}
data['artist_full_name'] = data['artist_full_name'].str.strip()
for name in data['artist_full_name'].unique():
    if name in tate_names:
        tate_natio[name] = str(tate_artists[tate_artists['name'] == name]['placeOfBirth'].values[0]).lower()
        counter += 1
print(counter)
tate_natio

0


C:\Users\linda\Miniconda3\envs\env_torch\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


{}

cmoa

In [60]:
cmoa_artists = pd.read_csv('cmoa.csv')
# MoMa_artists['DisplayName'] = MoMa_artists['DisplayName'].str.lower()
# MoMa_artists['DisplayName'] = MoMa_artists['DisplayName'].str.strip()
# cmoa_artists['name_split'] = cmoa_artists['name'].str.split(',')
# cmoa_artists['name'] = cmoa_artists['name'].str.split(',').str[1]+' '+cmoa_artists['name'].str.split(',').str[0]
# cmoa_artists['name'] = cmoa_artists['name'].str.lower()
cmoa_artists['full_name'] = cmoa_artists['full_name'].str.lower()
cmoa_artists['nationality'] = cmoa_artists['nationality'].str.lower()

In [61]:
cmoa_names = list(cmoa_artists['full_name'])
counter = 0
cmoa_natio = {}
data['artist_full_name'] = data['artist_full_name'].str.strip()
for name in data['artist_full_name'].unique():
    if name in cmoa_names:
        cmoa_natio[name] = str(cmoa_artists[cmoa_artists['full_name'] == name]['nationality'].values[0]).lower()
        counter += 1
print(counter)
cmoa_natio

C:\Users\linda\Miniconda3\envs\env_torch\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


4


{'jan luyken': 'dutch',
 'theodor de bry': 'flemish',
 'richard blome': 'english',
 'jacques callot': 'french'}

Create dict of {nationality:country}

In [7]:
country_replace = pd.concat([countries['Name'].str.lower(), countries['Nationality'].str.lower()], axis=1)
country_replace = country_replace.set_index('Name')
replace_name_nat = country_replace.to_dict()['Nationality']
replace_nat_name = {replace_name_nat[name]:name for name in list(replace_name_nat)}
# Remove values where key and values are same
# keys = list(replace_nat_name)
# values = [replace_nat_name[key] for key in keys]
# set(keys)&set(values)
del replace_name_nat['christmas island']
del replace_name_nat['djibouti']
del replace_name_nat['gibraltar']
del replace_nat_name['christmas island']
del replace_nat_name['djibouti']
del replace_nat_name['gibraltar']

In [8]:
replace_nat_name

{'andorran': 'andorra',
 'emirati': 'united arab emirates',
 'afghan': 'afghanistan',
 'antiguan, barbudan': 'antigua and barbuda',
 'anguillian': 'anguilla',
 'albanian': 'albania',
 'armenian': 'armenia',
 'dutch': 'netherlands',
 'angolan': 'angola',
 '?': 'wake island',
 'argentinean': 'argentina',
 'american samoan': 'american samoa',
 'austrian': 'austria',
 'australian': 'australia',
 'aruban': 'aruba',
 'swedish': 'sweden',
 'azerbaijani': 'azerbaijan',
 'bosnian, herzegovinian': 'bosnia and herzegovina',
 'barbadian': 'barbados',
 'bangladeshi': 'bangladesh',
 'belgian': 'belgium',
 'burkinabe': 'burkina faso',
 'bulgarian': 'bulgaria',
 'bahraini': 'bahrain',
 'burundian': 'burundi',
 'beninese': 'benin',
 'saint barthélemy islander': 'saint barthélemy',
 'bermudian': 'bermuda',
 'bruneian': 'brunei',
 'bolivian': 'bolivia',
 'brazilian': 'brazil',
 'bahamian': 'bahamas',
 'bhutanese': 'bhutan',
 'motswana': 'botswana',
 'belarusian': 'belarus',
 'belizean': 'belize',
 'canad

In [9]:
data = resultbig
data = data[data['iconclass_codes'].notna()]
# len(data[data['iconclass_codes'].isna()]), len(data[data['iconclass_codes'].notna()])

In [10]:
data = data[data['country'].isin(names)]

KeyError: 'country'

In [ ]:
data['country'] = data['artist_full_name'].str.lower()
data['ctry_id'] = data['country']


# TODO: add names of other datasets here
names = list(moma_natio)

# Remove entries with names not in the dict
data = data[data['country'].isin(names)]
data = data[data['country']!='unknown']

# Replace names to nationality
data = data.replace({"country": moma_natio}) 

# Replace nationality to country
data = data.replace({"country": replace_nat_name}) 

# Replace country to ctry_id
data['ctry_id'] = data['country']
data = data.replace({"ctry_id": country_dict_lower}) 

data


In [ ]:
len(data['iconclass_codes'].unique())

In [16]:
len(set([moma_natio[key] for key in list(moma_natio)]))

50

array(['hans von aachen', 'juan de la abadia', "niccol? dell' abbate",
       ..., 'clare, mavis.',
       'villeneuve-bargemont, louis francois de - marquis de villeneuve-trans',
       'tenot, eugene.'], dtype=object)

In [29]:
moma_names = list(MoMa_artists['DisplayName'])
counter = 0
moma_natio = {}
for name in data['artist_full_name'].unique():
    if name in moma_names:
        moma_natio[name] = MoMa_artists[MoMa_artists['DisplayName'] == name]['Nationality'].values[0]
        counter += 1
print(counter)
moma_natio

1157


{'albert besnard': 'French',
 'ford madox brown': 'British',
 'edward burne-jones': 'British',
 'mary cassatt': 'American',
 'lovis corinth': 'German',
 'henri-edmond cross': 'French',
 'george cruikshank': 'British',
 'albert dubois-pillet': 'French',
 'james ensor': 'Belgian',
 'henri evenepoel': 'French',
 'henri fantin-latour': 'French',
 'jean-louis forain': 'French',
 'akseli gallen-kallela': 'Finnish',
 'paul gauguin': 'French',
 'paul gavarni': 'French',
 'vincent van gogh': 'Dutch',
 'spencer frederick gore': 'British',
 'childe hassam': 'American',
 'louis hayet': 'French',
 'ferdinand hodler': 'Swiss',
 'max klinger': 'German',
 'georges lemmen': 'Belgian',
 'max liebermann': 'German',
 'maximilien luce': 'French',
 'claude monet': 'French',
 'berthe morisot': 'French',
 'william morris': 'British',
 'hippolyte petitjean': 'French',
 'camille pissarro': 'French',
 'lucien pissarro': 'French',
 'jan preisler': 'Czech',
 'odilon redon': 'French',
 'pierre-auguste renoir': 'Fre

In [18]:
MoMa_artists

,ConstituentID,DisplayName,ArtistBio,Nationality,Gender,BeginDate,EndDate,Wiki QID,ULAN
0,1,robert arneson,"American, 1930–1992",American,Male,1930,1992,NaN,NaN
1,2,doroteo arnaiz,"Spanish, born 1936",Spanish,Male,1936,0,NaN,NaN
2,3,bill arnold,"American, born 1941",American,Male,1941,0,NaN,NaN
3,4,charles arnoldi,"American, born 1946",American,Male,1946,0,Q1063584,500027998.0
4,5,per arnoldi,"Danish, born 1941",Danish,Male,1941,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
15710,132165,eric klarenbeek,"Dutch, born 1978",Dutch,Male,1978,0,NaN,NaN
15711,132166,maartje dros,Dutch,Dutch,Female,0,0,NaN,NaN
15712,132170,the event horizon telescope collaboration,NaN,NaN,NaN,0,0,NaN,NaN
15713,132427,krasnyi proletarii,NaN,NaN,NaN,0,0,NaN,NaN


In [3]:
counter = 0
with open(filename, 'rb') as f:
    for line in f:
        print(line)
        if counter == 10:
            break
        counter +=1

b'\xff\xfe*\x00 \x00E\x00x\x00a\x00m\x00p\x00l\x00e\x00 \x00g\x00e\x00n\x00e\x00r\x00a\x00t\x00e\x00d\x00 \x00b\x00y\x00 \x00-\x00d\x00a\x00t\x00a\x00e\x00x\x00-\x00.\x00 \x00T\x00o\x00 \x00i\x00n\x00s\x00t\x00a\x00l\x00l\x00:\x00 \x00s\x00s\x00c\x00 \x00i\x00n\x00s\x00t\x00a\x00l\x00l\x00 \x00d\x00a\x00t\x00a\x00e\x00x\x00\r\x00\n'
b'\x00c\x00l\x00e\x00a\x00r\x00\r\x00\n'
b'\x00i\x00n\x00p\x00u\x00t\x00 \x00s\x00t\x00r\x004\x002\x00 \x00C\x00o\x00u\x00n\x00t\x00r\x00y\x00N\x00a\x00m\x00e\x00O\x00r\x00i\x00g\x00 \x00s\x00t\x00r\x003\x000\x00 \x00C\x00o\x00u\x00n\x00t\x00r\x00y\x00N\x00a\x00m\x00e\x00S\x00t\x00a\x00n\x00d\x00a\x00r\x00d\x00 \x00s\x00t\x00r\x003\x00 \x00C\x00T\x00R\x00Y\x00C\x00O\x00D\x00E\x00 \x00s\x00t\x00r\x003\x000\x00 \x00D\x00 \x00s\x00t\x00r\x001\x004\x00 \x00E\x00c\x00o\x00n\x00o\x00m\x00i\x00c\x00c\x00l\x00a\x00s\x00s\x00I\x00 \x00s\x00t\x00r\x005\x00 \x00E\x00c\x00o\x00n\x00o\x00m\x00i\x00c\x00c\x00l\x00a\x00s\x00s\x00I\x00I\x00 \x00s\x00t\x00r\x003\x004\x00 \x

In [55]:
def replace_regex(regexs, S):
    # Replace
    for regex in regexs:
        S.replace(regex[0], regex[1],regex=True, inplace = True)
    return S

replace_regex(regexs, S)

0            Fin
1            Fin
2        Florida
3          Japan
4    Puerto Rico
5         Russia
6         france
dtype: object

In [19]:
data_meta['school'].value_counts()

unknown                                 662420
london                                  380278
modern                                  184195
paris                                    79474
new york                                 37413
                                         ...  
brabant                                      1
printed for j. tonson; and m. tonson         1
pariz                                        1
canton                                       1
watford                                      1
Name: school, Length: 4428, dtype: int64

In [31]:
counter = 0

freq = dict(data_meta['school'].value_counts())
for key in list(freq):
    print(key, freq[key])
    if counter == 5:
        break
    counter +=1


unknown 662420
london 380278
modern 184195
paris 79474
new york 37413
british 34348


In [ ]:
# To Remove:
remove = ['unknown', 'modern']

# Correct:
correct = ['london', 'paris']

replace = []

In [30]:
# del freq['unknown']
freq

KeyError: 'unknown'

In [53]:
# Remove the dash(-) followed by number from all countries in the Series
S=pd.Series(['Finland-1','finland-2','Florida-3','Japan-4','Puerto Rico-5','Russia-6','france-7'])
S.replace('(-\d)','',regex=True, inplace = True)

# regex
regexs = [(re.compile(r'finland', flags=re.IGNORECASE), 'Fin')]

def replace_regex(regexs, S):
    # Replace
    for regex in regexs:
        S.replace(regex[0], regex[1],regex=True, inplace = True)
    return S

replace_regex(regexs, S)
S

0            Fin
1            Fin
2        Florida
3          Japan
4    Puerto Rico
5         Russia
6         france
dtype: object